# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [55]:
#Your code here
# import needed libraries
import pandas as pd
import numpy as np
# load file into dataframe
df = pd.read_csv('homepage_actions.csv')
# view data
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [56]:
exp = df[df['group'] == 'experiment']
ctrl = df[df['group'] == 'control']

print('Control Group:', ctrl['action'].value_counts())
print('Experimental Group:', exp['action'].value_counts())

print((932/3332) *100, ' percent of the control group clicked')
print((928/2996) *100, 'percent of the experimental group clicked')
print("That's an increase of {}%" .format((.3097463284379172 - .2797118847539016) * 100) )

Control Group: view     3332
click     932
Name: action, dtype: int64
Experimental Group: view     2996
click     928
Name: action, dtype: int64
27.971188475390157  percent of the control group clicked
30.97463284379172 percent of the experimental group clicked
That's an increase of 3.0034443684015644%


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [57]:
df['count'] = 1

In [58]:
control = df[df.group=='control'].pivot(index='id', columns = 'action', values = 'count')
control.fillna(0, inplace =True)

In [59]:
experiment = df[df.group == 'experiment'].pivot(index='id', columns= 'action', values = 'count')
experiment.fillna(0, inplace=True)

In [60]:
import scipy.stats as stats
# conduct Welch's t-test
result = stats.ttest_ind(control['click'], experiment['click'], equal_var=False)

In [61]:
# divide p-value in half to get 1 tailed p-value (the experiment is to see if the control is higher, not different)
result[1]/2

0.004466402814337101

In [62]:
# With a p-value that low reject the null hypothesis

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [63]:
#Your code here
con_click_rate = control['click'].mean()
print(experiment['view'].sum() * con_click_rate)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [64]:
#Your code here
n = len(experiment['view'])
p = con_click_rate
var = n * p * (1-p)
np.sqrt(var)

24.568547907005815

In [67]:
z_score = (experiment['click'].sum() - 838.0168067226891) / 24.568547907005815
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [70]:
#Your code here
1 - stats.norm.cdf(z_score)

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: The p-value calculated from the z-score is lower than that of the statistical test; both p-values indicate that the 
likelihood of obtaining that increase in clicks of the sample group due to chance is extremely low. Reject the null hypothesis. The change in the webpage seems to have been effective.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.